# Métrica 1.3.03: "Congruencia en el número de declaraciones presentadas"

In [4]:
import re
import urllib.parse
from pymongo import MongoClient, ASCENDING, DESCENDING
from pymongo.errors import ConnectionFailure, OperationFailure
import unicodedata
import traceback
from datetime import datetime
from collections import defaultdict

# Importamos desde tu config
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

# --- 1. CONFIGURACIÓN BÁSICA ---
METRIC_ID_ESCOLARIDAD = "1_3_01_CONGRUENCIA_ESCOLARIDAD"
METRIC_ID_SECUENCIA = "1_3_02_SECUENCIA_TIPO"
METRIC_ID_CONTEO = "1_3_03_CONGRUENCIA_CONTEO"  # [NUEVO 1.3.03]

# --- 2. LÓGICA PURA DE LAS MÉTRICAS ---

# --- Lógica 1.3.01 (Escolaridad) ---
LEVEL_RANK_MAP = {
    "PRI": 1, "SEC": 2, "BCH": 3, "CTC": 3,
    "LIC": 4, "ESP": 5, "MTR": 5, "DOC": 6
}

def parse_iso_date(date_val) -> datetime:
    """
    Convierte un valor de fecha (string ISO o datetime) a un objeto datetime.
    """
    if isinstance(date_val, datetime):
        return date_val # Ya es un datetime
    if not isinstance(date_val, str): 
        return None
    try:
        # Usamos [:-1] para quitar la 'Z' al final
        return datetime.strptime(date_val[:-1], "%Y-%m-%dT%H:%M:%S")
    except (ValueError, TypeError):
        return None

def get_max_escolaridad(escolaridad_array: list) -> int:
    max_rank = 0
    if not isinstance(escolaridad_array, list): return 0
    for estudio in escolaridad_array:
        if isinstance(estudio, dict) and estudio.get("estatus") == "FINALIZADO":
            rank = LEVEL_RANK_MAP.get(estudio.get("nivel", {}).get("clave"))
            if rank and rank > max_rank:
                max_rank = rank
    return max_rank

# --- Lógica 1.3.02 (Secuencia) ---
def normalizar_tipo(tipo_str: str) -> str:
    if not isinstance(tipo_str, str):
        return "DESCONOCIDO"
    
    tipo_str = tipo_str.upper().strip()
    nfkd_form = unicodedata.normalize('NFD', tipo_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('ASCII')

# --- [NUEVO 1.3.03] Lógica 1.3.03 (Congruencia Conteo) ---
def parse_simple_date(date_str: str) -> datetime:
    """
    [NUEVO 1.3.03]
    Convierte un string de fecha simple (YYYY-MM-DD) a un objeto datetime.
    """
    if not isinstance(date_str, str): return None
    try:
        return datetime.strptime(date_str, "%Y-%m-%d")
    except (ValueError, TypeError):
        return None

def get_fecha_toma_posesion(doc: dict) -> datetime:
    """
    [NUEVO 1.3.03]
    Función helper para extraer la fecha de toma de posesión de un documento.
    """
    try:
        # Path basado en tu ejemplo: declaracion.situacionPatrimonial.datosEmpleoCargoComision.fechaTomaPosesion
        fecha_str = doc.get("declaracion", {})\
                       .get("situacionPatrimonial", {})\
                       .get("datosEmpleoCargoComision", {})\
                       .get("fechaTomaPosesion")
        
        return parse_simple_date(fecha_str)
    except Exception:
        return None

# --- 3. EL FLUJO DEL SCRIPT INDIVIDUAL ---
def procesar_historial_individual():
    """
    Pide un nombre completo, busca su historial, LO AGRUPA POR INSTITUCIÓN,
    y aplica las Métricas 1.3.01, 1.3.02 y 1.3.03
    dentro de cada grupo.
    """
    client = None
    
    # --- A. OBTENER DATOS DEL USUARIO ---
    print("--- Análisis Individual de Métrica Longitudinal (1.3.01, 1.3.02, 1.3.03) ---")
    print("Por favor, introduce el nombre completo del declarante a procesar:")
    
    nombre = input("Nombre(s) (ej. ALEJANDRA): ").upper().strip()
    paterno = input("Primer Apellido (ej. HERNANDEZ): ").upper().strip()
    materno_input = input("Segundo Apellido (ej. PEREZ) (deja en blanco si no aplica): ").upper().strip()
    
    path_nombre = "declaracion.situacionPatrimonial.datosGenerales.nombre" 
    path_paterno = "declaracion.situacionPatrimonial.datosGenerales.primerApellido"
    path_materno = "declaracion.situacionPatrimonial.datosGenerales.segundoApellido"
    path_fecha = "metadata.actualizacion"

    # --- B. CONSTRUIR FILTRO ROBUSTO ---
    filtro_busqueda = {
        path_nombre: nombre,
        path_paterno: paterno,
    }
    
    if materno_input == "":
        filtro_busqueda[path_materno] = { "$in": [None, ""] }
        print(f"\nBuscando historial para: {nombre} {paterno} (SIN APELLIDO MATERNO)")
    else:
        filtro_busqueda[path_materno] = materno_input
        print(f"\nBuscando historial para: {nombre} {paterno} {materno_input}")
    
    try:
        # --- C. CONECTAR Y ASEGURAR ÍNDICE ---
        print(f"Conectando a MongoDB en {DB_NAME}...")
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        client.admin.command('ping') 
        db = client[DB_NAME]
        source_collection = db[SOURCE_COLLECTION_NAME]
        target_collection = db[METRICS_COLLECTION_NAME]
        
        print("¡Conexión exitosa!")
        
        print("Asegurando el índice longitudinal...")
        source_collection.create_index([
            (path_nombre, ASCENDING),
            (path_paterno, ASCENDING),
            (path_materno, ASCENDING),
            (path_fecha, DESCENDING) 
        ])
        print("     > ¡Índice asegurado!")

        # --- D. BUSCAR HISTORIAL Y AGRUPAR POR INSTITUCION ---
        
        # [NUEVO 1.3.03] Añadimos el path de fechaTomaPosesion a la proyección
        proyeccion = {
            "_id": 1, "id": 1, 
            "metadata.actualizacion": 1, 
            "metadata.institucion": 1,
            "metadata.tipo": 1,
            "declaracion.situacionPatrimonial.datosCurricularesDeclarante.escolaridad": 1,
            # [NUEVO 1.3.03] Campo necesario para la métrica 1.3.03
            "declaracion.situacionPatrimonial.datosEmpleoCargoComision.fechaTomaPosesion": 1
        }
        
        historial_cursor = source_collection.find(filtro_busqueda, proyeccion)
        
        historial_por_institucion = defaultdict(list)
        
        for doc in historial_cursor:
            fecha_obj = parse_iso_date(doc.get("metadata", {}).get("actualizacion"))
            if fecha_obj:
                doc["fecha_obj"] = fecha_obj
                institucion = doc.get("metadata", {}).get("institucion", "SIN_INSTITUCION_DEFINIDA")
                historial_por_institucion[institucion].append(doc)
                
        if not historial_por_institucion:
            print(f"--- NO SE ENCONTRARON REGISTROS ---")
            return

        print(f"Se encontraron declaraciones en {len(historial_por_institucion)} institución(es) distintas.")
        
        if len(historial_por_institucion) > 1:
            print("Se procesará el historial de cada institución por separado.")

        # --- E. PROCESAR EL HISTORIAL POR CADA INSTITUCIÓN ---
        resultados_guardados_totales = 0
        
        for institucion, historial_institucion in historial_por_institucion.items():
            
            print(f"\n--- Procesando Institución: {institucion} ({len(historial_institucion)} registros) ---")
            
            # Ordenamos por fecha de actualización (metadata)
            historial_institucion.sort(key=lambda x: x["fecha_obj"])
            
            # [NUEVO 1.3.03] Buscamos el año de toma de posesión (x_2)
            # Recorremos la historia (ya ordenada) y tomamos la *primera* fecha de posesión que encontremos.
            ano_toma_posesion_inicial = None 
            for doc in historial_institucion:
                fecha_posesion = get_fecha_toma_posesion(doc)
                if fecha_posesion:
                    ano_toma_posesion_inicial = fecha_posesion.year
                    break # Encontramos la primera, es la que necesitamos
            
            if ano_toma_posesion_inicial:
                print(f"     > Año de toma de posesión (x_2) detectado: {ano_toma_posesion_inicial}")
            else:
                print(f"     > ADVERTENCIA: No se encontró 'fechaTomaPosesion' para esta institución. 1.3.03 será 'SIN_DATO'.")

            # Estado inicial para la métrica de secuencia
            estado_secuencia = "ESPERA_INICIAL"
            
            for i in range(len(historial_institucion)):
                doc_actual = historial_institucion[i]
                id_actual = doc_actual["_id"]
                string_id = doc_actual.get("id", "N/D")
                tipo_bruto = doc_actual.get("metadata", {}).get("tipo")
                tipo_actual = normalizar_tipo(tipo_bruto)
                
                metricas_a_guardar = {}
                doc_anterior = historial_institucion[i-1] if i > 0 else None
                
                # --- MÉTRICA 1: CONGRUENCIA ESCOLARIDAD (1.3.01) ---
                resultado_metrica_escolaridad = ""
                
                if not doc_anterior:
                    resultado_metrica_escolaridad = "N/A" # Es la primera declaración
                else:
                    escolaridad_actual = doc_actual.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosCurricularesDeclarante", {}).get("escolaridad")
                    escolaridad_anterior = doc_anterior.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosCurricularesDeclarante", {}).get("escolaridad")
                    x_2_esc = get_max_escolaridad(escolaridad_actual)
                    x_1_esc = get_max_escolaridad(escolaridad_anterior)

                    if x_1_esc == 0 or x_2_esc == 0:
                        resultado_metrica_escolaridad = "SIN_DATO"
                    else:
                        diferencia = x_2_esc - x_1_esc
                        if 0 <= diferencia <= 1:
                            resultado_metrica_escolaridad = "CUMPLE"
                        else:
                            resultado_metrica_escolaridad = "NO_CUMPLE"
                
                metricas_a_guardar[METRIC_ID_ESCOLARIDAD] = resultado_metrica_escolaridad
                
                
                # --- MÉTRICA 2: SECUENCIA DE TIPO (1.3.02) ---
                resultado_metrica_secuencia = ""
                
                if estado_secuencia == "ESPERA_INICIAL":
                    if tipo_actual == "INICIAL":
                        resultado_metrica_secuencia = "SECUENCIA_CORRECTA"
                        estado_secuencia = "ESPERA_MODIFICACION_O_CONCLUSION"
                    else:
                        resultado_metrica_secuencia = "ERROR_REVISION_INICIAL_FALTANTE"
                        estado_secuencia = "ERROR"
                elif estado_secuencia == "ESPERA_MODIFICACION_O_CONCLUSION":
                    if tipo_actual == "MODIFICACION":
                        resultado_metrica_secuencia = "SECUENCIA_CORRECTA"
                    elif tipo_actual == "CONCLUSION":
                        resultado_metrica_secuencia = "SECUENCIA_CORRECTA"
                        estado_secuencia = "FINALIZADO"
                    elif tipo_actual == "INICIAL":
                        resultado_metrica_secuencia = "ERROR_SECUENCIA_DOBLE_INICIAL"
                        estado_secuencia = "ERROR"
                    else:
                        resultado_metrica_secuencia = "ERROR_TIPO_DESCONOCIDO"
                        estado_secuencia = "ERROR"
                elif estado_secuencia == "FINALIZADO":
                    resultado_metrica_secuencia = "ERROR_SECUENCIA_POST_CONCLUSION"
                    estado_secuencia = "ERROR"
                elif estado_secuencia == "ERROR":
                    resultado_metrica_secuencia = "ERROR_SECUENCIA_PREVIA"
                else:
                    resultado_metrica_secuencia = "ERROR_LOGICA_DESCONOCIDA"

                metricas_a_guardar[METRIC_ID_SECUENCIA] = resultado_metrica_secuencia

                
                # --- [NUEVO 1.3.03] MÉTRICA 3: CONGRUENCIA CONTEO (1.3.03) ---
                resultado_metrica_conteo = ""
                
                # x_1 = Número de declaraciones presentadas (i es 0-index, +1 para conteo)
                x_1 = i + 1 
                # x_2 = Año de toma de posesión (calculado arriba)
                x_2 = ano_toma_posesion_inicial 
                # x_3 = Último año declarado (el año de esta declaración)
                x_3 = doc_actual['fecha_obj'].year
                
                if x_2 is None:
                    # No tenemos el año de posesión, no podemos calcular
                    resultado_metrica_conteo = "SIN_DATO"
                elif x_2 < x_3:
                    # Condición: "Si el declarante se desempeñó... el año anterior"
                    # Es decir, el año de posesión es ANTERIOR al año actual
                    
                    # Aplicamos la fórmula: x_1 + x_2 - x_3 ≥ 1
                    if (x_1 + x_2 - x_3) >= 1:
                        # 0: Cumple con el criterio
                        resultado_metrica_conteo = "CUMPLE"
                    else:
                        # 1: No cumple con el criterio
                        resultado_metrica_conteo = "NO_CUMPLE"
                else:
                    # El año de posesión es el mismo que el de la declaración (x_2 >= x_3)
                    # Es su primer año, la métrica no aplica.
                    resultado_metrica_conteo = "N/A"

                metricas_a_guardar[METRIC_ID_CONTEO] = resultado_metrica_conteo

                
                # --- F. GUARDAR RESULTADOS (Todas las métricas) ---
                print(f"\n   Procesando Documento ID: {id_actual} (id: {string_id})")
                print(f"     > Fecha: {doc_actual['fecha_obj'].date()}, Tipo: {tipo_actual}")
                print(f"     > Métrica Escolaridad (1.3.01): {resultado_metrica_escolaridad}")
                print(f"     > Métrica Secuencia (1.3.02): {resultado_metrica_secuencia}")
                print(f"     > Métrica Conteo (1.3.03): {resultado_metrica_conteo} (x1={x_1}, x2={x_2}, x3={x_3})") # [NUEVO 1.3.03]
                
                filtro = { "_id": id_actual }
                actualizacion = { "$set": metricas_a_guardar } # Guarda todas
                target_collection.update_one(filtro, actualizacion, upsert=True)
                print(f"     > ¡Resultados guardados/actualizados en 'metricas'!")
                resultados_guardados_totales += 1

        print("\n--- PROCESAMIENTO INDIVIDUAL FINALIZADO ---")
        print(f"Se procesaron y guardaron {resultados_guardados_totales} resultados en total.")

    except ConnectionFailure: print("Error: No se pudo conectar a la base de datos.")
    except OperationFailure as e: print(f"Error en la operación de la base de datos: {e.details}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        traceback.print_exc()
    finally:
        if client: client.close(); print("Conexión cerrada.")

# --- 4. EJECUTAR EL SCRIPT ---
if __name__ == "__main__":
    procesar_historial_individual()

--- Análisis Individual de Métrica Longitudinal (1.3.01, 1.3.02, 1.3.03) ---
Por favor, introduce el nombre completo del declarante a procesar:

Buscando historial para: ALEJANDRA HERNANDEZ PEREZ
Conectando a MongoDB en sistema1...
¡Conexión exitosa!
Asegurando el índice longitudinal...
     > ¡Índice asegurado!
Se encontraron declaraciones en 7 institución(es) distintas.
Se procesará el historial de cada institución por separado.

--- Procesando Institución: INSTITUTO DE EDUCACIÓN DE AGUASCALIENTES (GOBIERNO DEL ESTADO DE AGUASCALIENTES) (11 registros) ---
     > Año de toma de posesión (x_2) detectado: 2023

   Procesando Documento ID: 68f81b8800535f910a2a0d33 (id: 190110)
     > Fecha: 2024-05-30, Tipo: INICIAL
     > Métrica Escolaridad (1.3.01): N/A
     > Métrica Secuencia (1.3.02): SECUENCIA_CORRECTA
     > Métrica Conteo (1.3.03): NO_CUMPLE (x1=1, x2=2023, x3=2024)
     > ¡Resultados guardados/actualizados en 'metricas'!

   Procesando Documento ID: 68f81b8800535f910a2a0d35 (id